In [94]:
# General Stuff:
import numpy as np
import pandas as pd
import os

# Our Stuff:
from models.AutoRecBase import AutoRecBase
from models.VarAutoRec import VarAutoRec
from models.MF import MF
from models.UserBiasAE import UserBiasAE


from scripts.get_data import download_2_data_sets, ratings_to_train_test, ratings_to_train_test_u
from scripts.get_2_other_data import get_2_other_datasets, secondary_to_train_test, secondary_to_train_test_u
from utils.evaluate import evaluate_model
from utils.loading_utils import load_model, save_model

import torch
from torch import nn
import pytorch_lightning as pl

# Visualization Stuff
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 240)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1660 Ti


# AutoRec - AutoEncoders Meet Collaborative Filtering - PyTorch

<a id="toc"></a>
## Table of Content
1. [Introduction](#introduction)
1. [Conclusions](#conclusions)

<a id="introduction"></a>

## Introduction

In this notebook we will review a collaborative filtering approach using autoencoders, as suggested by Sedhain et al, in their 2015 paper "AutoRec: Autoencoders Meet Collaborative Filtering".

We will then introduce several improvements and asses them.

[Table of content](#toc)

In [95]:
is_default_dataset = False
if is_default_dataset:
    download_2_data_sets()
else:
    get_2_other_datasets()

In [96]:
if is_default_dataset:
    train_loader, val_loader = ratings_to_train_test(1,0, 1,10)
    mf_train_loader, mf_val_loader =  ratings_to_train_test_u(dataset_size=1,
                                                              validation_partition=0,
                                                              train_partition=1,
                                                              batch_size=10)
else:
    train_loader, val_loader = secondary_to_train_test('douban',0, 1,10)
    mf_train_loader, mf_val_loader =  secondary_to_train_test_u('douban',
                                                                validation_partition=0,
                                                                train_partition=1,
                                                                batch_size=10)


In [24]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...

Go to:  [TensorBoard](http://localhost:6006)

In [ ]:
models_dict = {} # (model,ephoc,lr): loss
models_state = {} # (model,ephoc,lr): model.state_dict()
models = [
    UserBiasAE
]
lrs = [0.001,0.005,0.01]
activations = [nn.PReLU, nn.Sigmoid]

Sanity check:

In [98]:
for x, y, m in val_loader:
    print(x[:,0])
    break
for x, y, m in mf_val_loader:
    print(x[0])
    break
for x, y, r in mf_train_loader:
    print(x)
    print(y)
    print(r)
    break
# model(x)[:,0]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor(0)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1617, 1494, 1395, 1391, 1387, 1706, 1324, 1184, 1178, 1177])
tensor([2., 2., 3., 4., 1., 4., 1., 2., 3., 1.], dtype=torch.float64)


In [99]:
print(f"Number of users: {len(val_loader.dataset)}")
print(f"Number of items: {len(val_loader.dataset[0][1])}")

Number of users: 2769
Number of items: 3000


Datasets:

In [100]:
number_of_users = len(val_loader.dataset)
number_of_items = len(val_loader.dataset[0][1])

From the original paper:

In [101]:
latent_dims = [10, 80, 300]
lambdas = [0.01, 0.1, 1, 100]

In [ ]:
should_train = True
model_paths = []

if should_train:
    for model_class in models:
        for lr in lrs:
            for latent in latent_dims:
                for activation in activations:
                    for λ in lambdas:
                        model = model_class(number_of_items=number_of_items,
                                            num_of_users=number_of_users,
                                            hidden_size=latent,
                                            activation_function_1=activation,
                                            activation_function_2=activation,
                                            loss=nn.MSELoss(reduction='none'),
                                            λ=λ,
                                            lr=lr)
                        # training
                        trainer = pl.Trainer(gpus=0, max_epochs=10)
                        try:
                            if type(model).__name__ == "MF":
                                trainer.fit(model,mf_train_loader, mf_val_loader)
                            else:
                                trainer.fit(model,train_loader, val_loader)

                            model_path = save_model(model_class=model_class,
                                       trainer=trainer,
                                       activation=activation,
                                       hidden_size=latent,
                                       lr=lr,
                                       λ=λ,
                                       is_default_dataset=is_default_dataset)
                            model_paths.append(model_path)
                        except:
                            pass
        #                 break
        #             break
        #         break
        #     break
        # break
print(model_paths)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/ido/anaconda3/envs/rs/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 30.0 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 33.0 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
63.0 K    Trainable params
0         Non-trainable params
63.0 K    Total params
0.252     Total estimated model params size (MB)


0.01
                                                              

/home/ido/anaconda3/envs/rs/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 0:  50%|█████     | 277/554 [00:01<00:01, 146.48it/s, loss=2.1, v_num=506] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 277/554 [00:01<00:01, 151.94it/s, loss=0.781, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 277/554 [00:01<00:01, 158.09it/s, loss=0.741, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 277/554 [00:01<00:01, 154.53it/s, loss=0.718, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 277/554 [00:01<00:01, 154.69it/s, loss=0.723, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 277/554 [00:01<00:01, 156.41it/s, loss=0.799, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 277/554 [00:01<00:01, 159.30it/s, loss=0.737, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 277/554 [00:01<00:01, 158.16it/s, loss=0.748, v_num=506]
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 277/554 [00:01<00:01, 152.74it/s, loss=0.775, v_num=506]
Valida

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 30.0 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 33.0 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
63.0 K    Trainable params
0         Non-trainable params
63.0 K    Total params
0.252     Total estimated model params size (MB)


...

Epoch 0:  50%|█████     | 277/554 [00:03<00:03, 73.77it/s, loss=0.825, v_num=791]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 277/554 [00:03<00:03, 74.33it/s, loss=0.875, v_num=791] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 277/554 [00:03<00:03, 73.35it/s, loss=0.864, v_num=791] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 277/554 [00:03<00:03, 74.88it/s, loss=0.842, v_num=791] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 277/554 [00:04<00:04, 65.61it/s, loss=0.83, v_num=791]  
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 277/554 [00:04<00:04, 65.34it/s, loss=0.863, v_num=791]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 277/554 [00:04<00:04, 65.73it/s, loss=0.831, v_num=791] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 277/554 [00:04<00:04, 68.86it/s, loss=0.889, v_num=791] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 277/554 [00:04<00:04, 68.98it/s, loss=0.857, v_num=791] 
Validat

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 900 K 
1 | act_1     | Sigmoid | 0     
2 | decoder   | Linear  | 903 K 
3 | act_2     | Sigmoid | 0     
4 | loss_func | MSELoss | 0     
--------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.213     Total estimated model params size (MB)


1
Epoch 0:  50%|█████     | 277/554 [00:03<00:03, 70.06it/s, loss=0.898, v_num=792]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 277/554 [00:03<00:03, 70.34it/s, loss=0.899, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 277/554 [00:03<00:03, 71.00it/s, loss=0.894, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 277/554 [00:03<00:03, 71.86it/s, loss=0.882, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 277/554 [00:03<00:03, 70.76it/s, loss=0.835, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 277/554 [00:03<00:03, 69.81it/s, loss=0.908, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 277/554 [00:04<00:04, 60.85it/s, loss=0.871, v_num=792] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 277/554 [00:03<00:03, 69.99it/s, loss=0.85, v_num=792]  
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 277/554 [00:04<00:04, 63.99it/s, loss=0.864, v_num=792]
Valid

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 900 K 
1 | act_1     | Sigmoid | 0     
2 | decoder   | Linear  | 903 K 
3 | act_2     | Sigmoid | 0     
4 | loss_func | MSELoss | 0     
--------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.213     Total estimated model params size (MB)


100
Epoch 0:  50%|█████     | 277/554 [00:04<00:04, 65.54it/s, loss=0.87, v_num=793] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 277/554 [00:04<00:04, 63.75it/s, loss=0.852, v_num=793]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 277/554 [00:04<00:04, 65.98it/s, loss=0.878, v_num=793] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 277/554 [00:04<00:04, 65.37it/s, loss=0.873, v_num=793] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 277/554 [00:04<00:04, 65.10it/s, loss=0.851, v_num=793] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 277/554 [00:04<00:04, 65.99it/s, loss=0.871, v_num=793] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  34%|███▍      | 191/554 [00:02<00:05, 65.45it/s, loss=0.848, v_num=793] 

In [85]:
# def attr_from_path(path):
#     class_name,activation,hidden_size,lr,λ,data_set_name = path.split(os.sep)[-1].replace('_model_dict.ckpt','').split('_')
#     return class_name,activation,hidden_size,lr,λ,data_set_name
#
# models_from_string = {'AutoRecBase':AutoRecBase,
#            'MF':MF,
#            'UserBiasAE':UserBiasAE,
#            'VarAutoRec':VarAutoRec}
#
# def load_model_2(model_class, activation, hidden_size, lr, λ, is_default_dataset):
#     path = f"obj/{model_class}_<class 'torch.nn.modules.{activation}'>_{hidden_size}_{lr}_{λ}_{is_default_dataset}_model_dict.ckpt"
#     model = models_from_string[model_class].load_from_checkpoint(path)
#     model.eval()
#     return model

In [79]:
# from glob import glob

In [ ]:
# obs = [path for path in glob('obj/*.ckpt') if 'True' in path]
# models_eval_dict = {}
# Ks = [10,20,50, 100]
# i = 0
#
#
# for model_path in obs:
#     model_class,activation,latent,lr,λ,data_set_name = attr_from_path(model_path)
#     print(model_path)
#     model = models_from_string[model_class].load_from_checkpoint(model_path)
#     print('loaded')
#     for K in Ks:
#         (hits, ndcgs, mrrs) = evaluate_model(model, test_loader=val_loader, K=K)
#         models_eval_dict[f"row_{i}"] = [type(model).__name__, activation, latent, λ, lr, K, "HR", np.mean(hits)]
#         models_eval_dict[f"row_{i+1}"] = [type(model).__name__, activation, latent, λ, lr, K, "NDCG",np.mean(ndcgs)]
#         models_eval_dict[f"row_{i+2}"] = [type(model).__name__, activation, latent, λ, lr, K, "MRR",np.mean(mrrs)]
#         i += 3
#
# columns = ["model", "activation", "latent_dim", "lambda", "lr","topk","metric","score"]
# eval_df = pd.DataFrame.from_dict(models_eval_dict, orient='index', columns=columns)
# eval_df['dataset'] = 'ml-1m'
# eval_df.to_csv("eval/eval_df_ml1.csv", sep='\t')
#
# eval_df.head()


In [ ]:
models_eval_dict = {}
Ks = [10,20,50, 100]
i = 0

for model_class in models:
    for activation in activations:
        for lr in lrs:
            for latent in latent_dims:
                for λ in lambdas:
                    try:
                        model = load_model(model_class=model_class,
                                           activation=activation,
                                           hidden_size=latent,
                                           lr=lr,
                                           λ=λ,
                                           is_default_dataset=is_default_dataset)
                        for K in Ks:
                            (hits, ndcgs, mrrs) = evaluate_model(model, test_loader=val_loader, K=K)
                            models_eval_dict[f"row_{i}"] = [type(model).__name__, activation, latent, λ, lr, K, "HR", np.mean(hits)]
                            models_eval_dict[f"row_{i+1}"] = [type(model).__name__, activation, latent, λ, lr, K, "NDCG",np.mean(ndcgs)]
                            models_eval_dict[f"row_{i+2}"] = [type(model).__name__, activation, latent, λ, lr, K, "MRR",np.mean(mrrs)]
                            i += 3
                    except Exception as e:
                        print(e)
    #                     break
    #                 break
    #             break
    #         break
    #     break
    # break

In [ ]:
columns = ["model", "activation", "latent_dim", "lambda", "lr","topk","metric","score"]
eval_df = pd.DataFrame.from_dict(models_eval_dict, orient='index', columns=columns)
eval_df['dataset'] = 'douban'
eval_df.to_csv("eval/eval_df_douban.csv", sep='\t')

eval_df.head()